In [136]:
import pandas as pd
import numpy as np
import math
from math import e
from scipy.stats import beta, norm, bernoulli
from numpy import linalg
from sklearn.linear_model import LogisticRegression
import warnings

warnings.filterwarnings("ignore")

In [137]:
data = pd.read_csv("athletes_edited.csv")
display(data)

,nationality,sport,age,male,height,weight,won_any,won_gold,total_medals
0,ESP,athletics,46,1,172,64,0,0,0
1,KOR,fencing,29,0,168,56,0,0,0
2,CAN,athletics,24,1,198,79,1,0,1
3,MDA,taekwondo,25,1,183,80,0,0,0
4,NZL,cycling,25,1,181,71,0,0,0
...,...,...,...,...,...,...,...,...,...
10853,CUB,athletics,20,0,164,58,0,0,0
10854,CZE,athletics,29,0,173,63,0,0,0
10855,CHN,wrestling,25,1,185,100,0,0,0
10856,VIE,weightlifting,27,1,160,56,0,0,0


# Part 1

### From project 2

**The research question -- Does the average weight (X) of the athletes change between the athletes who won a medal (Y=1) and the ones who didn't (Y=0)?**

## Q1

In [138]:
# we set random_state in order to get consistent results
sampled_data = data.sample(n=200, random_state=0)
sampled_indecies = sampled_data.index.values

data_without_sampled = data.drop(sampled_indecies, axis=0)
past_data = data_without_sampled.sample(n=1000, random_state=0)

display(sampled_data)
display(past_data)

,nationality,sport,age,male,height,weight,won_any,won_gold,total_medals
6798,GER,athletics,29,1,189,78,0,0,0
2124,CUB,athletics,28,0,171,60,0,0,0
159,NIG,judo,28,1,180,73,0,0,0
4148,EGY,judo,34,1,195,105,0,0,0
1478,RSA,aquatics,25,1,158,86,0,0,0
...,...,...,...,...,...,...,...,...,...
3000,HUN,aquatics,30,0,173,64,0,0,0
1835,AUS,aquatics,17,0,162,62,0,0,0
2818,RUS,aquatics,22,0,166,62,1,0,1
9643,CIV,aquatics,21,0,176,63,0,0,0


,nationality,sport,age,male,height,weight,won_any,won_gold,total_medals
8283,GBR,canoe,27,0,177,70,0,0,0
3394,NZL,sailing,22,0,165,59,0,0,0
8475,CHI,archery,16,1,183,88,0,0,0
7793,UKR,aquatics,21,0,178,61,0,0,0
7474,USA,aquatics,27,1,199,102,1,1,4
...,...,...,...,...,...,...,...,...,...
8209,KAZ,gymnastics,18,1,170,63,0,0,0
7471,SWE,handball,25,0,167,62,0,0,0
10260,RUS,shooting,46,1,178,83,0,0,0
4140,JPN,athletics,28,1,165,54,0,0,0


## Q2

We will choose $\tau$ to be the weight median of sampled_data

In [139]:
tau = sampled_data["weight"].median()
print(f"tau value: {tau}")

tau value: 68.5


### Section A

We will estimate the log risk ratio using plug-in estimator.

We will find a confidence interval for log risk ratio using the bootstrap percentile method.

In [140]:
won_weight = sampled_data[sampled_data["won_any"] == 1]["weight"]
not_won_weight = sampled_data[sampled_data["won_any"] == 0]["weight"]

# the number of iterations in each simulation
B = 500

# the confidende level of the confidence intervals is 0.95
alpha = 0.05


def eta(p):
    return math.log(p / (1 - p))


def find_psi_hat(won_weight, not_won_weight):
    p1_hat = sum([1 if x > tau else 0 for x in won_weight]) / len(won_weight)
    p2_hat = sum([1 if x > tau else 0 for x in not_won_weight]) / len(not_won_weight)
    log_risk_ratio_hat = eta(p1_hat) - eta(p2_hat)

    return log_risk_ratio_hat


def psi_bootstrap(won_weight, not_won_weight, B):
    n, m = len(won_weight), len(not_won_weight)
    bootstrap_lst = []

    for b in range(B):
        # we set random_state in order to get consistent results
        won_weight_b = won_weight.sample(n=n, random_state=b, replace=True)
        not_won_weight_b = not_won_weight.sample(n=m, random_state=b, replace=True)

        psi_hat_b = find_psi_hat(won_weight_b ,not_won_weight_b)
        bootstrap_lst.append(psi_hat_b)

    return sorted(bootstrap_lst)

psi_bootstrap_lst = psi_bootstrap(won_weight, not_won_weight, B)

In [141]:
# find the estimator of log risk ratio based on sampled_data
psi_hat = find_psi_hat(won_weight, not_won_weight)

# find the confidence interval of log risk ratio with confidence level of 0.95 (as stated in the project)
low_q = psi_bootstrap_lst[int((alpha / 2) * B)]
high_q = psi_bootstrap_lst[int((1 - (alpha / 2)) * B)]
confidence_interval = np.array([low_q, high_q])

print("--- frequentistic ---")
print(f"log risk ratio estimator: {round(psi_hat, 3)}")
print(f"confidence interval for log risk ratio: {np.round(confidence_interval, 3)}")

--- frequentistic ---
log risk ratio estimator: 0.285
confidence interval for log risk ratio: [-0.454  1.103]


### Section B

Notations:

- Let $Z_1$ be the same definition as $Z$, but refers only to the athletes' weight who won a medal. Note that $Z_1 \sim Bernoulli(p_1)$

- Let $n$ be the number of athletes who won a medal (from sampled data).

- Let $S_1 = \sum_{i=1}^{n}Z^{i}_{1}$

- Let $Z_2$ be the same definition as $Z$, but refers only to the athletes' weight who didn't win a medal. Note that $Z_2 \sim Bernoulli(p_2)$

- Let $m$ be the number of athletes who didn't win any medal (from sampled data).

- Let $S_2 = \sum_{i=1}^{m}Z_{2}^{i}$

First, let us define prior distributions for our parameters $p_1$ and $p_2$, as follows:

$p_1, p_2 \sim Uni[0, 1]$

We saw in the lecture that if we have a uniform prior and the data parametric model is Bernoulli, then the posterior is $Beta(S+1, \space n-S+1)$

thus the posteriors of $p_1$ and $p_2$ are as follows:

$p_1 | Z_1^n \sim Beta(S_1+1, \space n-S_1+1)$

$p_2 | Z_2^m \sim Beta(S_2+1, \space m-S_2+1)$


In [142]:
n, m = len(won_weight), len(not_won_weight)
s1 = sum([1 if x > tau else 0 for x in won_weight])
s2 = sum([1 if x > tau else 0 for x in not_won_weight])


def psi_simulate(a1, b1, a2, b2, B):
    simulate_lst = []

    for b in range(B):
        # we set random_state in order to get consistent results
        p1_b = beta.rvs(a1, b1, random_state=b)
        p2_b = beta.rvs(a2, b2, random_state=b)

        psi_b = eta(p1_b) - eta(p2_b)
        simulate_lst.append(psi_b)

    return sorted(simulate_lst)

In [143]:
a1, b1 = s1+1, n-s1+1
a2, b2 = s2+1, m-s2+1

psi_simulate_lst = psi_simulate(a1, b1, a2, b2, B)

# approximate the expectation using simulation as shown in the course's book
psi_mean_approx = sum(psi_simulate_lst) / B

# find credible interval as shown in the course's book
low_q = psi_simulate_lst[int((alpha / 2) * B)]
high_q = psi_simulate_lst[int((1 - (alpha / 2)) * B)]
credible_interval = np.array([low_q, high_q])

print("--- bayesian - unifrom prior ---")
print(f"log risk ratio estimator: {round(psi_mean_approx, 3)}")
print(f"credible interval for log risk ratio: {np.round(credible_interval, 3)}")

--- bayesian - unifrom prior ---
log risk ratio estimator: 0.273
credible interval for log risk ratio: [-0.068  0.641]


## Section C

First, we will define the prior distributions for our parameters to be Jeffreys' prior.

We saw in the lecture that when the data parametric model is Bernoulli then the prior is as follows: $\sqrt{1 / p(1-p)}$

Thus, the priors for $p_1$ and $p_2$ are $\sqrt{1 / p_1(1-p_1)}$ and $\sqrt{1 / p_2(1-p_2)}$ correspondingly.

After calculation **(*)**, we found that the posterior distributions for $p_1$ and $p_2$ are as follows:

$p_1 | Z_1^n \sim Beta(S_1+\frac{1}{2}, \space n-S_1+\frac{1}{2})$

$p_2 | Z_2^m \sim Beta(S_2+\frac{1}{2}, \space m-S_2+\frac{1}{2})$

$\space$

Calculation **(*)**:

For $p_1 | Z_1^n$:

### $f(p_1 | Z_1^n) \propto L_n(p_1) \pi(p_1) = \sqrt{\frac{1}{p_1(1-p_1)}} * p_1^{S_1}(1-p_1)^{n-S_1} = p_1^{S_1-\frac{1}{2}} (1 - p_1)^{n-S_1-\frac{1}{2}} = p_1^{(S_1+\frac{1}{2})-1} (1 - p_1)^{(n-S_1+\frac{1}{2})-1}$

As we can see, the density function we found is that of a Beta distribution with the parameters mentioned above (up to a constant).

The same calculation applies to $p_2 | Z_2^m$ as well. Hence, $p_2 | Z_2^m$ is distributed Beta with the parameters above.

In [144]:
a1, b1 = s1 + 0.5, n - s1 + 0.5
a2, b2 = s2 + 0.5, m - s2 + 0.5

psi_simulate_lst = psi_simulate(a1, b1, a2, b2, B)

# approximate the expectation using simulation as shown in the course's book
psi_mean_approx = sum(psi_simulate_lst) / B

# find credible interval as shown in the course's book
low_q = psi_simulate_lst[int((alpha / 2) * B)]
high_q = psi_simulate_lst[int((1 - (alpha / 2)) * B)]
credible_interval = np.array([low_q, high_q])

print("--- bayesian - Jeffreys' prior ---")
print(f"log risk ratio estimator: {round(psi_mean_approx, 3)}")
print(f"credible interval for log risk ratio: {np.round(credible_interval, 3)}")

--- bayesian - Jeffreys' prior ---
log risk ratio estimator: 0.28
credible interval for log risk ratio: [-0.07   0.658]


### Section D

As approved in the forum, we will do the following:

First, we will assume $Uni(0, \space 1)$ priors for $p_1$ and $p_2$.

Then we will find the posterior based on the past data. as mentioned in section B, the posteriors will be:

$p_1 | Z_1^n \sim Beta(S_1^{past}+1, \space n^{past}-S_1^{past}+1)$, where $S_1^{past}$ and $n^{past}$ are based on the past data.

$p_2 | Z_2^m \sim Beta(S_2^{past}+1, \space m^{past}-S_2^{past}+1)$, where $S_2^{past}$ and $m^{past}$ are based on the past data.

Now, we will refer to the posteriors based on the past data as our priors to $p_1$ and $p_2$ when dealing with 'sampled_data' (the priors we got are indeed in the Beta distribution family as mentioned in the question).

As we saw in the lecture, $Beta(\alpha, \space \beta)$ and $Bernoulli(p)$ classes are conjugate, with the posterior of the form $Beta(\alpha + S, \space \beta + n - S)$

Thus, the final posteriors will be as follows:

$p_1 | Z_1^n \sim Beta([S_1^{past}+1] + [S_1^{new}], \space [n^{past}-S_1^{past}+1] + [n^{new}-S_1^{new}])$, where $S_1^{new}$ and $n^{new}$ are based on the 'sampled_data'.

$p_2 | Z_2^m \sim Beta([S_2^{past}+1] + [S_2^{new}], \space [m^{past}-S_2^{past}+1] + [m^{new}-S_2^{new}])$, where $S_2^{new}$ and $m^{new}$ are based on the 'sampled_data'.

In [145]:
# find the posteriors of p_1 and p_2 beta parameters 
past_data_won_weight = past_data[past_data["won_any"] == 1]["weight"]
past_data_not_won_weight = past_data[past_data["won_any"] == 0]["weight"]

n_past, m_past = len(past_data_won_weight), len(past_data_not_won_weight)
n_new, m_new = len(won_weight), len(not_won_weight)

s1_past = sum([1 if x > tau else 0 for x in past_data_won_weight])
s2_past = sum([1 if x > tau else 0 for x in past_data_not_won_weight])

s1_new = sum([1 if x > tau else 0 for x in won_weight])
s2_new = sum([1 if x > tau else 0 for x in not_won_weight])

# the parameters of the posteriors of p1, p2
a1 = (s1_past + 1) + (s1_new)
b1 = (n_past - s1_past + 1) + (n_new - s1_new)

a2 = (s2_past + 1) + (s2_new)
b2 = (m_past - s2_past + 1) + (m_new - s2_new)

In [146]:
psi_simulate_lst = psi_simulate(a1, b1, a2, b2, B)

# approximate the expectation using simulation as shown in the course's book
psi_mean_approx = sum(psi_simulate_lst) / B

# find credible interval as shown in the course's book
low_q = psi_simulate_lst[int((alpha / 2) * B)]
high_q = psi_simulate_lst[int((1 - (alpha / 2)) * B)]
credible_interval = np.array([low_q, high_q])

print("--- bayesian - Beta distribution family ---")
print(f"log risk ratio estimator (bayesian) {round(psi_mean_approx, 3)}")
print(f"credible interval for log risk ratio: {np.round(credible_interval, 3)}")

--- bayesian - Beta distribution family ---
log risk ratio estimator (bayesian) 0.228
credible interval for log risk ratio: [0.082 0.386]


### Section E

In section A, we estimated the log risk ratio using the frequentistic approach. On the other hand, in sections B-D we estimated the log risk ratio using the Bayesian approach.

- In section A, we chose a plug-in estimator and the estimator's value is 0.285

- In section B, we chose uniform priors $Uni(0, \space 1)$ for $p_1$ and $p_2$, and the estimator's value is 0.273

- In section C, we chose Jeffreys' prior for $p_1$ and $p_2$, and the estimator's value is 0.28

- In section D, we chose priors from $Beta$ distribution family for $p_1$ and $p_2$, and the estimator's value is 0.228

As we can see, the estimators we got in all sections are very similar, though the estimator in section A uses a different approach from the estimators in sections B to D. Moreover, the estimators in sections B and C assume different priors. In addition, the estimator in section D make use of past data (the procedure is described in section D).

To conclude, all the methods produced similar results, but not identical. Therefore, choosing a statistical approach (frequentistic or Bayesian) and choosing a prior may influence the results.

# Part 2

The explaining variables we chose:
- height (continuous) - represents the height (in cm) of the athlete. This variable has many values, and therefore we can consider it as a continuous variable by the project instructions.

- age (continuous) - represents the age (in years) of the athlete. This variable has many values, and therefore we can consider it as a continuous variable by the project instructions.

- male (discrete) - represents the gender of the athlete. This variable has only two values, 1 if the athlete is male, and 0 if the athlete is female. It is a binary variable and therefore discrete.

The explained variable we chose:
- weight (continuous) - represents the weight (in kg) of the athlete. This variable has many values, and therefore we can consider it as a continuous variable by the project instructions.

(*) this part was taken from project 3.

In [147]:
# functions from project 3 we will use in this project

def extract_X_y(data, X_columns, y_column):
    # create X matrix with first column as bias column
    X = data[X_columns]
    X.insert(loc=0, column="bias", value=[1 for i in range(len(X))])
    X = X.to_numpy()

    # create y vector
    y = data[y_column].to_numpy()

    return X, y


def find_beta_hat(X, y):
    beta_hat = linalg.inv(X.T @ X) @ X.T @ y
    return beta_hat


def find_sum_squares(beta_hat, X, y):
  y_avg = np.mean(y)
  SSR, SSE = 0, 0
  for xi, yi in zip(X, y):
    y_hat = beta_hat.T @ xi
    SSR += (y_hat - y_avg) ** 2
    SSE += (y_hat - yi) ** 2

  SST = SSR + SSE

  return SSR, SSE, SST


def find_var_hat(beta_hat, X, y):
    _, SSE, _ = find_sum_squares(beta_hat, X, y)
    n, p = X.shape
    MSE = SSE / (n - p)

    return MSE * linalg.inv(X.T @ X)


def find_confidence_intervals(beta_hat, var_hat):
  z = norm.ppf(0.975)

  for i, beta_i_hat in enumerate(beta_hat):
    se_i_hat = var_hat[i][i] ** 0.5
    confidence_interval = [round(beta_i_hat - (z * se_i_hat), 3), round(beta_i_hat + (z * se_i_hat), 3)]
    print(f"confidence interval for beta {i}:")
    print(confidence_interval)
    print()

## Q1

As mentioend in projcet 1, we removed all records with missing values. Thus, we can just sample from our data.

In [148]:
# we set random_state in order to get consistent results
sampled_data2 = data.sample(n=1000, random_state=0)

## Q2

In [149]:
X, y = extract_X_y(sampled_data2, X_columns=["height", "age", "male"], y_column="weight")
beta_hat = find_beta_hat(X, y)

print("---- Full data ----")
print("Mean square error estimator for beta (based on sampled_data2):")
print(list(np.around(beta_hat, 3)))

---- Full data ----
Mean square error estimator for beta (based on sampled_data2):
[-98.876, 0.939, 0.076, 6.197]


In [150]:
_, SSE, _ = find_sum_squares(beta_hat, X, y)
n, p = X.shape
MSE = SSE / (n-p)

var_hat = MSE * linalg.inv(X.T @ X)

print("confidence intervals using the entire 'sampled_data2'\n")
find_confidence_intervals(beta_hat, var_hat)

confidence intervals using the entire 'sampled_data2'

confidence interval for beta 0:
[-110.418, -87.335]

confidence interval for beta 1:
[0.871, 1.006]

confidence interval for beta 2:
[-0.047, 0.199]

confidence interval for beta 3:
[4.695, 7.7]



## Q3

Although our y variable has repeated values, there are very few repetitions, so we were permitted to refer to our values as unique (approved in the question forum).

In [151]:
# find indicies to remove
remove_indicies = []

for i in range(1000):
    # the higher the index of the record, the higher the y value
    # therefore the higher the probabilty to remove the y value
    p_i = i / 1000

    # we set random_state in order to get consistent results
    if bernoulli.rvs(p_i, random_state=i):
        remove_indicies.append(i)

# create the missing values data
y_sorted = sorted(y)
y_removed = []

for i, val in enumerate(y_sorted):

    if i in remove_indicies:
        y_removed.append(None)
        continue
    
    y_removed.append(val)

sampled_data2_removed = sampled_data2.sort_values("weight")
sampled_data2_removed["weight"] = y_removed

display(sampled_data2_removed)
X_removed, y_removed = extract_X_y(sampled_data2_removed, X_columns=["height", "age", "male"], y_column="weight")

,nationality,sport,age,male,height,weight,won_any,won_gold,total_medals
10460,CHN,gymnastics,16,0,140,33.0,1,0,1
10562,CHN,gymnastics,16,0,151,35.0,1,0,1
188,JPN,gymnastics,16,0,146,35.0,0,0,0
7198,JPN,athletics,20,0,154,39.0,0,0,0
2228,BRA,gymnastics,31,0,147,40.0,0,0,0
...,...,...,...,...,...,...,...,...,...
7014,USA,athletics,30,0,176,NaN,1,1,1
1466,ESP,athletics,32,1,204,NaN,0,0,0
1136,SRB,athletics,31,1,187,NaN,0,0,0
10521,TUR,shooting,29,1,185,NaN,0,0,0


## Q4

### Section A

In [152]:
# remove all records with missing values
sampled_data2_complete = sampled_data2_removed.dropna()
X_complete, y_complete = extract_X_y(sampled_data2_complete, X_columns=["height", "age", "male"], y_column="weight")

beta_hat_complete = find_beta_hat(X_complete, y_complete)
var_hat_complete = find_var_hat(beta_hat_complete, X_complete, y_complete)

print("based on the complete data\n")

print("Mean square error estimator for beta:")
print(list(np.around(beta_hat_complete, 3)))

print("\nconfidence intervals\n")
find_confidence_intervals(beta_hat_complete, var_hat_complete)

based on the complete data

Mean square error estimator for beta:
[-62.268, 0.729, -0.039, 4.688]

confidence intervals

confidence interval for beta 0:
[-74.286, -50.249]

confidence interval for beta 1:
[0.658, 0.799]

confidence interval for beta 2:
[-0.153, 0.074]

confidence interval for beta 3:
[3.294, 6.081]



### Section B

In [153]:
# find the regression imputations of the missing values
y_with_reg_imputations = []

for xi, yi in zip(X_removed, y_removed):

    # if yi is missing, impute its values using the linear regression prediction
    if np.isnan(yi):
        yi_pred = beta_hat_complete.T @ xi
        y_with_reg_imputations.append(yi_pred)
        continue

    y_with_reg_imputations.append(yi)

sampled_data_reg_imputation = sampled_data2_removed.copy()
sampled_data_reg_imputation["weight"] = y_with_reg_imputations

In [154]:
X_reg_imputation, y_reg_imputation = extract_X_y(sampled_data_reg_imputation, X_columns=["height", "age", "male"], y_column="weight")

beta_hat_reg_imputation = find_beta_hat(X_reg_imputation, y_reg_imputation)
var_hat_reg_imputation = find_var_hat(beta_hat_reg_imputation, X_reg_imputation, y_reg_imputation)

print("based on regression imputation \n")

print("Mean square error estimator for beta (based on regression imputation):")
print(list(np.around(beta_hat_reg_imputation, 3)))

print("\nconfidence intervals\n")
find_confidence_intervals(beta_hat_reg_imputation, var_hat_reg_imputation)

based on regression imputation 

Mean square error estimator for beta (based on regression imputation):
[-62.268, 0.729, -0.039, 4.688]

confidence intervals

confidence interval for beta 0:
[-67.695, -56.84]

confidence interval for beta 1:
[0.697, 0.76]

confidence interval for beta 2:
[-0.097, 0.019]

confidence interval for beta 3:
[3.981, 5.394]



Notation:

- Let $\hat \beta_{Complete}$ be the mean square estimator based on the complete data (removing all missing data).
- Let $\hat \beta_{Reg \_ imputation}$ be the mean square estimator based on regression imputation.

As we can see, $\hat \beta_{Reg \_ imputation} = \hat \beta_{Complete}$

$\hat \beta_{Complete}$ is trivially optimal for the complete data. In addition, we impute the missing values using the regression estimation of $\hat \beta_{Complete}$. Therefore, the error of this estimator on the imputed values is 0 by the definition of the imputation. Thus, it is the optimal estimator for the imputed data as well.


Moreover, the confidence intervals based on the two methods are different. One of the reasons is that the number of records in each method is different (the number of records under the regression imputation method is bigger since we consider the missing values as well [after imputation]).
 

### Section C + D

In this section, we will use multiple imputation methods as seen in the lecture.

We saw in the lecture that the MI estimator is distributed asymptotically Normal. Therefore, we can use the Normal approximation confidence interval.

(*) We will assume that y values are distributed $N(\beta^{*T}X, \space \sigma^2_\epsilon)$ as stated in the question (where $\sigma^2_\epsilon$ is the variance of the noise $\epsilon$).

(**) We will define $M = 100$

In [155]:
M = 100

def multiple_imputation(X, y, beta_hat, std, M):
    beta_hat_lst, var_hat_lst = [], []

    for m in range(M):
        y_new_m = []

        for xi, yi in zip(X, y):
            
            # if the y value is missing we impute by sampling
            if np.isnan(yi):
                mean_i = beta_hat.T @ xi
                # we set random_state in order to get consistent results
                yi_pred = norm.rvs(loc=mean_i, scale=std, random_state=m)
                y_new_m.append(yi_pred)
                continue
            
            y_new_m.append(yi)

        beta_hat_m = find_beta_hat(X, y_new_m)
        var_hat_m = find_var_hat(beta_hat_m, X, y_new_m)

        beta_hat_lst.append(beta_hat_m)
        var_hat_lst.append(var_hat_m)

    return beta_hat_lst, var_hat_lst


def get_var_hat_beta_hat_MI(beta_hat_lst, var_hat_lst, beta_hat_MI, M):
    # calculates variance using Robin's formula (while replacing Fisher's information
    # with the variance^(-1))

    var_hat_beta_hat_MI = []
    for i, beta_hat_MI_i in enumerate(beta_hat_MI):
        item1, item2 = 0, 0

        for beta_hat, var_hat in zip(beta_hat_lst, var_hat_lst):
            item1 += var_hat[i][i]
            item2 += (beta_hat[i] - beta_hat_MI[i]) ** 2

        item1 *= (1 / M)
        item2 *= (M+1) / (M * (M - 1))
        var_i_hat = item1 + item2
        var_hat_beta_hat_MI.append(var_i_hat)

    return var_hat_beta_hat_MI


_, SSE_complete, _ = find_sum_squares(beta_hat_complete, X_complete, y_complete)
n_complete, p_complete = X_complete.shape
MSE_complete = SSE_complete / (n_complete - p_complete)


beta_hat_lst , var_hat_lst = multiple_imputation(X_removed, y_removed, beta_hat_complete, 
                                                 MSE_complete ** 0.5, M)

In [156]:
beta_hat_MI = sum(beta_hat_lst) / len(beta_hat_lst)

var_hat_beta_hat_MI = get_var_hat_beta_hat_MI(beta_hat_lst, var_hat_lst, beta_hat_MI, M)
temp = np.diag(var_hat_beta_hat_MI)

print("based on Multiple Imputation\n")

print("Mean square error estimator:")
print(list(np.around(beta_hat_MI, 3)))

print("\nconfidence intervals:\n")
find_confidence_intervals(beta_hat_MI, temp)

based on Multiple Imputation

Mean square error estimator:
[-62.762, 0.732, -0.039, 4.711]

confidence intervals:

confidence interval for beta 0:
[-103.979, -21.545]

confidence interval for beta 1:
[0.469, 0.995]

confidence interval for beta 2:
[-0.112, 0.034]

confidence interval for beta 3:
[2.604, 6.818]



### Section E

In this section, we will estimate $P(R=1|X_1, X_2, X_3)$ using logistic regression where $X_1, X_2, X_3$ are the explaning variables ('height', 'age', 'male').

In order to achieve it, we will find the MLE estimator.

In [157]:
def sigmoid(x):
    return (e ** x) / (1 + e ** x)

In [158]:
# creating the R colmun: 0 if the data is missing, else 1
def add_R_column(df):
    df_LR = df.copy()

    R_lst = []
    for y in df_LR["weight"]:
        if np.isnan(y):
            R_lst.append(0)
        
        else:
            R_lst.append(1)

    df_LR["R"] = R_lst

    return df_LR

df_LR = add_R_column(sampled_data2_removed)
X_LR, y_LR = extract_X_y(df_LR, X_columns=["height", "age", "male"], y_column="R")

# find the MLE estimator
reg = LogisticRegression(penalty="none", fit_intercept=False).fit(X_LR, y_LR)
beta_hat_LR = reg.coef_[0]

print("Logistic regression: \n")
print(f"The model MLE estimator:")
print(list(np.around(beta_hat_LR, 3)))

# the probability predictions of seeing the records
pi_lst = [sigmoid(beta_hat_LR.T @ xi) for xi in X_LR]

Logistic regression: 

The model MLE estimator:
[20.424, -0.113, -0.013, -0.593]


### Section F

Let us display the linear regression problem at hand as a Least Squares problem:

$LS: \space (y - X \beta)^T W (y - X \beta)$

Where $W$ is a diagonal matrix of size $n$ x $n$, and $W_{ii} = R_i / \pi _i$ for $1 \le i \le n$ (we already found $R_i$, $\pi _i$ for all $X_i$ , $1 \le i \le n$ in previous questions). Expanding the brackets, we get:

$LS: \space y^T W y - 2(X^T W y)^T \beta + (X \beta)^T W (X \beta)$

To find the LS estimator, we would like to minimize the LS optimization problem. So, we will take the derivative and set it equal to zero:

$-2 X^T W y + 2(X^T W X) \beta = 0$

$(X^T W X) \beta = X^T W y$

$\Rightarrow \hat \beta_{IPW} = (X^T W X)^{-1} X^T W y$

In [159]:
def find_beta_hat_IPW(X, y, W):
    beta_hat_IPW = linalg.inv(X.T @ W @ X) @ X.T @ W @ np.nan_to_num(y)
    return beta_hat_IPW

weight_lst = [(Ri / pi) for pi, Ri in zip(pi_lst, df_LR["R"])]
beta_hat_IPW = find_beta_hat_IPW(X_removed, y_removed, W=np.diag(weight_lst))

print("based on IPW \n")

print("Mean square error estimator:")
print(list(np.around(beta_hat_IPW, 3)))

based on IPW 

Mean square error estimator:
[-66.061, 0.747, -0.008, 4.357]


### Section G

In this section, we will find confidence intervals using beta hat IPW estimator and bootstrap percentile method.

In [160]:
def beta_hat_IPW_bootstrap(data_removed, B):
    n = len(data_removed)
    beta_hat_IPW_lst = []

    for b in range(B):
        # sample the bootstrap sample
        # we set random_state in order to get consistent results
        b_data_removed = data_removed.sample(n=n, replace=True, random_state=b)
        b_df_LR = add_R_column(b_data_removed)

        # perform logistic regression on the bootstrapped sample
        b_X_LR, b_y_LR = extract_X_y(b_df_LR, X_columns=["height", "age", "male"], y_column="R")
        b_reg = LogisticRegression(penalty="none", fit_intercept=False).fit(b_X_LR, b_y_LR)
        b_beta_hat_LR = b_reg.coef_[0]
        
        # find weight list
        b_weight_lst = [(Ri / sigmoid(b_beta_hat_LR.T @ xi)) for xi, Ri in zip(b_X_LR, b_df_LR["R"])]

        # extract linear regression columns
        b_X, b_y = extract_X_y(b_df_LR, X_columns=["height", "age", "male"], y_column="weight")

        # find beta hat IPW estimator (using the weight list found with logisitc regression)
        b_beta_hat_IPW = find_beta_hat_IPW(b_X, b_y, W=np.diag(b_weight_lst))
        beta_hat_IPW_lst.append(b_beta_hat_IPW)

    return beta_hat_IPW_lst

beta_hat_IPW_lst = beta_hat_IPW_bootstrap(sampled_data2_removed, B)

In [161]:
print("based on IPW\n")
    
for i in range(len(beta_hat_IPW)):
    beta_hat_i_IPW_lst = sorted([beta[i] for beta in beta_hat_IPW_lst])
    low_q = beta_hat_i_IPW_lst[int((alpha / 2) * B)]
    high_q = beta_hat_i_IPW_lst[int((1 - (alpha / 2)) * B)]
    
    confidence_interval = [round(low_q, 3), round(high_q, 3)]
    print(f"confidence interval for beta {i}:")
    print(confidence_interval)

    if i != len(beta_hat_IPW) - 1:
        print()

based on IPW

confidence interval for beta 0:
[-78.553, -53.596]

confidence interval for beta 1:
[0.669, 0.825]

confidence interval for beta 2:
[-0.154, 0.142]

confidence interval for beta 3:
[2.737, 5.938]


### Section H

In question 2, the estimator we got (before creating missing values), $\hat \beta = [-98.876, 0.939, 0.076, 6.197]$.

After finding this estimator, we were asked in question 3 to create missing values in our data.

In order to find an estimator to $\beta^*$ while dealing with the missing data, we used the following methods:

- Complete data (removing missing records) -- the estimator we got under this method $\hat \beta_{Complete} = [-62.268, 0.729, -0.039, 4.688]$.
  
- Regression imputation -- the estimator we got under this method $\hat \beta_{Reg\_imputation} = [-62.268, 0.729, -0.039, 4.688]$.

- Multiple imputation -- the estimator we got under this method $\hat \beta_{Multi\_imputation} = [-62.762, 0.732, -0.039, 4.711]$.

- IPW -- the estimator we got under this method $\hat \beta_{IPW} = [-66.061, 0.747, -0.008, 4.357]$.

As we can see, none of the estimators above are very similar to the estimator on the full data. However, we would still like to see which of the methods has produced the estimator that's closest to $\hat \beta$.

We will use Euclidean distance to compare the different estimators and select the one with the minimum Euclidean distance to $\hat \beta$).
    

In [162]:
method_estimator_lst = [("complete", beta_hat_complete),
                        ("reg. imputation", beta_hat_reg_imputation),
                        ("multi. imputation", beta_hat_MI),
                        ("IPW", beta_hat_IPW)]

best_method = min(method_estimator_lst, key=lambda tup: linalg.norm(beta_hat - tup[1]))[0]
print(f"The method that produced the closet estimator to beta_hat: {best_method}")

The method that produced the closet estimator to beta_hat : IPW


The confidence intervals we received under this part:

- Full data (before creating missing values):

    [-110.418, -87.335], $\space \space$ [0.871, 1.006], $\space \space$ [-0.047, 0.199], $\space \space$ [4.695, 7.7]

- Complete data (removing missing records):

    [-74.286, -50.249],$\space \space \space \space \space$ [0.658, 0.799], $\space \space$ [-0.153, 0.074], $\space \space$ [3.294, 6.081]

- Regression Imputation:

    [-67.695, -56.84], $\space \space \space \space \space \space$ [0.697, 0.76], $\space \space \space \space$ [-0.097, 0.019], $\space \space$ [3.981, 5.394]

- Multiple imputation:

    [-103.979, -21.545], $\space \space$ [0.469, 0.995], $\space \space$ [-0.112, 0.034], $\space \space$ [2.604, 6.818]

- IPW:

    [-78.553, -53.596], $\space \space \space \space$ [0.669, 0.825], $\space \space$ [-0.154, 0.142], $\space \space$ [2.737, 5.938]

As we can see, the confidence intervals we received using the methods of estimation with missing data are rather different from the confidence interval we got using the full data and are also different from each other. Therefore, we can conclude that missing data has an impact on finding confidence intervals as well as the method of dealing with it.

In our results:

The confidence intervals for $\beta_0, \beta_1, \beta_2, \beta_3$ under regression imputation are the shortest among all methods.

The confidence intervals for $\beta_0, \beta_1, \beta_3$ under multiple imputation are the longest among all methods.

The confidence interval for $\beta_2$ under IPW is the longest among all methods.